In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np


import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from gensim.utils import simple_preprocess, tokenize
from gensim.summarization.textcleaner import split_sentences
from gensim.parsing import preprocessing
from gensim.models import Doc2Vec
from gensim.corpora import Dictionary


from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from multiprocessing import cpu_count

# pd.set_option('display.max_colwidth',1000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

%matplotlib inline

cuda


In [2]:
with open('data/dev/cluster_1_pivot.pkl', 'rb') as f:
     samples, pivots = pickle.load(f)

In [3]:
df_sample = samples[0]
pivot_sample = pivots[0]

In [4]:
df_sample.head()

,id,category,body,time
0,3-8474601,luonto,Myös siemenpankki sijaitsee Huippuvuorilla. M...,0
1,3-6360914,luonto,Riskikohteista kaivataan tietoa. Suomen kansa...,1
2,3-6812190,luonto,Millaista apua kaipaisit susihavaintojen yhtey...,2
3,3-6309102,luonto,Herra Saimaa sai äänensä takaisin. Saimaan ve...,2
4,3-6986878,luonto,Ylitornio: Museoviraston valitus mäkikeskukses...,3


In [ ]:
class SeqDataset(Dataset):

    def __init__(self, df, wv=None, max_len=250):
        self.df = df
        self.wv = wv
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        vocab = self.wv.vocab
        doc_len = len(item['text'])
        
        # truncate to max len or add padding after
        if doc_len >= self.max_len:
            tokens = item['text'][:self.max_len]
        else:
            tokens = item['text'] + ['<pad>'] * (self.max_len - doc_len)
        
        token_ids = [vocab.get(t).index if t in vocab else 1 for t in tokens]
        
        token_ids = torch.tensor(token_ids, dtype=int)
        label = torch.tensor(item['label'], dtype=torch.float)

        return token_ids, label
